In [1]:
import sys
sys.path.append('/home/pi/Desktop/ADR_NX/Utils')
import serial,time,logging,adr_utils,Queue
from timeit import default_timer as timer 
debug = 1

In [8]:
def init_and_open_connection():
    ser = adr_utils.open_serial_port('/dev/ADR_sim900')
    ser.open()
    ser.write("*CLS\n")
    time.sleep(1)
    ser.flushInput()
    ser.flushOutput()

In [4]:
global sim900q_out, sim900q_in
sim900q_out = Queue.Queue(1000) #FIFO queue OUT
sim900q_in = Queue.Queue(1000) #FIFO queue IN
delaylist = ['*IDN','RPER']

# Adds command to the queue
# Types - 0=no response/reg wait; 1=imm response/reg wait; 2=imm response/extra wait
def oq_add(cmd, tp=1):
    if debug: print("QADD ({}): {}".format(tp,cmd))
    sim900q_out.put_nowait([cmd,tp])
    
def oq_process():
    start = timer()
    while (not sim900q_out.empty and (timer()-start)<10.0):
        [cmd, tp] = sim900q_out.get_nowait()
        if debug: print("QSEND ({}): {}".format(tp,cmd))
        ser.write(cmd)
        if (tp == 2):
            time.sleep(0.4)
        else:
            time.sleep(0.1)
        if (tp >= 1):
            continue
    
def iq_process():
    start = timer()
    while (not sim900q_in.empty and (timer()-start)<10.0):
        cmd = sim900q_in.get_nowait()
        
def iq_process_one():
    start = timer()   
    cmd = sim900q_in.get_nowait()
    if debug: print("QREC: {}".format(cmd))
    end = timer()
    
def msg_parser(msg):
    check1 = msd.startsWith("MSG")
    if not check1
        raise IOError("Bad message parse")
    msg = msg.substring(3,END)
    numdigit = msg.substring(1,2)
    if numdigit == '2'
        numchar = msg.substring(2,4)
    elif numdigit == '3'
        numchar = msg.substring(2,5)
    else:
        raise IOError("Bad message digit counter") 
    
    msg = msg.substring(2+int(numdigit),END)
    if len(msg) != numchar
        raise IOError("Bad message payload length") 
        
    return msg

SyntaxError: invalid syntax (<ipython-input-4-294ede9d7485>, line 38)

In [ ]:
def read_line(ser, tm=1, critical=False):
    tm_old = ser.timeout
    ser.timeout = tm
    resp = ser.readline().rstrip()
    ser.timeout = tm_old
    if (resp == '' and critical):
        raise IOError("Critical response not received")    
    return resp

#get identity
def get_IDN():
    oq_add(cmd_IDN())
    return read_line(ser, 1)
def cmd_IDN:
    return ('*IDN?\n')

#get message passthrough bits
def get_RPER():
    oq_add(cmd_RPER())
    return read_line(ser, 1)
def cmd_get_RPER:
    return ('RPER?\n')

#set message passthrough bits 
def set_RPER(ports):
    oq_add(cmd_RPER(ports))
def cmd_set_RPER(ports):
    portbin = 0
    for i in ports:
        portbin += int(1 << i)    
    return ('RPER {}\n'.format(portbin))   

#reset message passthrough bits    
def reset_RPER():
    oq_add(cmd_reset_RPER(port))  
def cmd_reset_RPER():
    return ('RPER 0\n')  
    
#send command to module  
def send_CMD_module(port, cmd):
    oq_add('SNDT {},"{}"\n'.format(port, cmd))

#send command to mainframe   
def send_CMD_mf(ser, cmd):
    ser.write('{}\n'.format(cmd))   

#baud change of mainframe
def set_baud_mf(ser):
    send_CMD_mf(ser, "BAUD D,115200")
                
#baud change of module   
def set_baud_module(ser, mod, rate):
    send_CMD_module(ser,mod,"BAUD {}".format(rate))
    send_CMD_mf(ser, "BAUD {},cmd")
    
#flush mainframe
def flush_mf(ser):
    send_CMD_mf(ser, "FLOQ")
    send_CMD_mf(ser, "FLSH")
    #ser.write("*CLS\n")

#flush our port
def flush_pi_port(ser):
    ser.flushInput()
    ser.flushOutput()
    
#reset comms on one of the ports
def reset_serial_module(ser, mod):
    send_CMD_module(ser,mod,"SRST {}".format(mod))

In [5]:
#enumerate all modules
def enumerate_mods(ser):
    for i in range(1,9):
        ser.write('SNDT {},"*IDN?"\n'.format(i))
        time.sleep(0.3)
        ser.write('GETN? {},128\n'.format(i))
        time.sleep(0.3)
        # have to not use readline because empty slots return weird results otherwise
        response = ser.read(128).encode('string_escape')
        print("Slot {} - {}".format(i,response))    

In [6]:
ser.write("*CLS\n")
ser.write("FLSH\n")
ser.timeout = 0.1
for i in range(1,9):
    ser.write('SNDT {},"*IDN?"\n'.format(i))
    time.sleep(0.1)
    ser.write('GETN? {},128\n'.format(i))
    time.sleep(0.1)
    response = ser.read(1024)
    print("Slot {} - {}".format(i,response.rstrip()))   

NameError: name 'ser' is not defined

In [11]:
ser = init_and_open_connection()

In [ ]:
string1 = 'CONN 6, "dde"\n'
ser.write(string1)

time.sleep(0.2)
response = ser.read(1024)
print 'R2:' + response

In [10]:
ser.write('*IDN?\n')
time.sleep(0.2)
response = ser.read(1024)
print 'R3:' + response

NameError: name 'ser' is not defined

In [ ]:
ser.read(1024)
maxt = maxt2 = 0
totalt = totalt2 = 0
for i in range(1,1000000):
    start = timer()
    ser.write('*IDN?\n')
    end = timer()
    #time.sleep(0.1)
    ser.timeout = 0.1
    response = ser.readline().rstrip();
    end2 = timer()
    delta = (end-start)
    delta2 = (end2-end)
    
    totalt += delta
    totalt2 += delta2
    if (maxt < delta):
        maxt = delta
    if (maxt2 < delta2):
        maxt2 = delta2
    if (response != "Stanford_Research_Systems,SIM960,s/n014669,ver2.17"):
        print("BROKE ON ITERATION {}, resp '{}'".format(i,response))
        break
    if (i % 10000 == 0):
        print("IT: {} | avg: {} | max: {} | avg2: {} | max2: {}".format(i,totalt/i,maxt,totalt2/i,maxt2))

In [ ]:
ser.write('CURV? 0\n')
time.sleep(0.2)
response = ser.read(1024)
print 'R3:' + response

In [ ]:
ser.write('CURV 0,1\n')
time.sleep(0.2)
response = ser.read(1024)
print 'R3:' + response

In [ ]:
ser.read(1024)
for i in range(1,100):
    ser.write("CAPT? 2,{}\n".format(i))
    time.sleep(0.4)
    response = ser.read(1024).rstrip()
    print "Point {}: {}".format(i,response)

In [ ]:
ser.write('TVAL? 0\n')
time.sleep(1)
response = ser.read(1024)
print 'R3:' + response

In [ ]:
ser.write('*RST\n')
time.sleep(0.2)
response = ser.read(1024)
print 'R3:' + response

In [ ]:
string2 = 'TVAL?' + str(4) + '\n'
ser.write(string2)

time.sleep(0.2)
response = ser.read(64)
print 'R4:' + response

In [ ]:
time.sleep(0.2)
ser.write('dde\n')

response = ser.read(64)
print 'R5:' + response

In [ ]:
ser.close()

In [ ]:
string1 = '*IDN?\n'
ser.write(string1)
time.sleep(2)
response = ser.read(64)
print response

In [ ]:
ser.write("*CLS\n")
ser.write("FLSH\n")
ser.timeout = 0.1
for i in range(1,9):
    ser.write('SNDT {},"*IDN?"\n'.format(i))
    time.sleep(0.1)
    ser.write('GETN? {},128\n'.format(i))
    time.sleep(0.1)
    response = ser.read(1024)
    print("Slot {} - {}".format(i,response.rstrip()))    

In [ ]:
ser.write("*CLS\n")
ser.write("FLSH\n")
ser.timeout = 0
for x in range(1000,50,-20):
    start = timer()
    ser.write('*IDN?\n')
    time.sleep(float(x)/1000)
    response = ser.read(128)
    end = timer()
    print('{} | {} | {}'.format(x,end - start,response).rstrip())

In [ ]:
ser.flushInput();
ser.write("*CLS\n")
ser.write("FLSH\n")
ser.timeout = 0.1
for x in range(1,20):    
    ser.write('*IDN?\n')
    start = timer()
    while ser.inWaiting == 0:
        pass
    end = timer()
    t1 = end - start
    response = ser.readline()
    end2 = timer()
    t2 = end2 - start
    print('{} | {} | {}'.format(t1,t2,response).rstrip())

In [ ]:
ser.write("FLSH\n")
for x in range(1000,50,-20):
    ser.write('SNDT 6,"TVAL?4"\n')
    time.sleep(float(x)/1000)
    ser.write('GETN? 6,128\n')
    time.sleep(float(x)/1000)
    response = ser.read(1024)
    ser.
    print('{} | {}'.format(x,response).rstrip())

In [ ]:
for (x * 0.1) in range(1000,50,-10):
    print(a)

In [ ]:
ser.write('SNDT 6,"TVAL?4"\n')
time.sleep(1)
ser.write('GETN? 6,128\n')
time.sleep(1)
response = ser.read(1024)
print response

In [ ]:
ser.write('TERM? 6,"TVAL?4"\n')
time.sleep(1)
ser.write('GETN? 6,128\n')
time.sleep(1)
response = ser.read(1024)
print response

In [ ]:
ser.write('SEND 6,"*RST\n"\n')
time.sleep(2)
response = ser.read(64)
print response

In [ ]:
ser.write('SRST 6\n')
time.sleep(2)
response = ser.read(64)
print response

In [ ]:
ser.write('RPER?\n')
time.sleep(2)
response = ser.read(64)
print response

In [ ]:
ser.write('TERM? 6\n')
time.sleep(2)
response = ser.read(64)
print response

In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )